In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # for progress bar
import logging

In [2]:
vehicles = pd.read_csv('vehicles.csv')
vehicles.head(10)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
5,7222379453,https://hudsonvalley.craigslist.org/cto/d/west...,hudson valley,https://hudsonvalley.craigslist.org,1600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ny,NaN,NaN,NaN
6,7221952215,https://hudsonvalley.craigslist.org/cto/d/west...,hudson valley,https://hudsonvalley.craigslist.org,1000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ny,NaN,NaN,NaN
7,7220195662,https://hudsonvalley.craigslist.org/cto/d/poug...,hudson valley,https://hudsonvalley.craigslist.org,15995,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ny,NaN,NaN,NaN
8,7209064557,https://medford.craigslist.org/cto/d/grants-pa...,medford-ashland,https://medford.craigslist.org,5000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,or,NaN,NaN,NaN
9,7219485069,https://erie.craigslist.org/cto/d/erie-2012-su...,erie,https://erie.craigslist.org,3000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,pa,NaN,NaN,NaN


In [3]:
vehicles.describe()

,id,price,year,odometer,county,lat,long
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05,0.0,420331.000000,420331.000000
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04,NaN,38.493940,-94.748599
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05,NaN,5.841533,18.365462
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-84.122245,-159.827728
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04,NaN,34.601900,-111.939847
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04,NaN,39.150100,-88.432600
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05,NaN,42.398900,-80.832039
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07,NaN,82.390818,173.885502


In [24]:
vehicles.columns


Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

Odometer

In [4]:
count, mean, std, min, twentyfive, fifty, seventyfive, max = vehicles['odometer'].describe() #get info on feature "odometer"
vehicles['odometer'].describe()

count    4.224800e+05
mean     9.804333e+04
std      2.138815e+05
min      0.000000e+00
25%      3.770400e+04
50%      8.554800e+04
75%      1.335425e+05
max      1.000000e+07
Name: odometer, dtype: float64

In [5]:
vehicles['odometer'].isna().sum() #Check number of NaN values

4400

In [6]:
vehicles['odometer'] = pd.to_numeric(vehicles['odometer'], errors='coerce') #ensure all values are numeric, and if therere are others they are turned into NaN
vehicles['odometer'].isna().sum() #Check number of NaN values

4400

In [7]:
vehicles = vehicles[vehicles['odometer'] < 5000000] #clip out any values above 5 000 000 miles

In [8]:
median_odometer = vehicles['odometer'].median() #fill all missing values with the median of remaning vehicles
vehicles['odometer'] = vehicles['odometer'].fillna(median_odometer)

In [9]:
#vehicles = vehicles[vehicles['odometer'].notna()] #option to remove all rows with NaN values, could be relevant since there are only 4400

In [10]:
newcount, newmean, newstd, newmin, newtwentyfive, newfifty, newseventyfive, newmax = vehicles['odometer'].describe() #get updated info on feature "odometer"
print('change in count: ', count-newcount)
print('change in mean: ', mean-newmean)
print('change in standard deaviation: ',std-newstd)
print('change in minimum: ', min-newmin)
print('change in maximum: ', max-newmax)
vehicles['odometer'].describe()

change in count:  187.0
change in mean:  4074.775355073798
change in standard deaviation:  127756.83104947585
change in minimum:  0.0
change in maximum:  5271776.0


count    4.222930e+05
mean     9.396856e+04
std      8.612467e+04
min      0.000000e+00
25%      3.768200e+04
50%      8.548300e+04
75%      1.334520e+05
max      4.728224e+06
Name: odometer, dtype: float64

Transmission

In [11]:
vehicles['transmission'].value_counts(dropna=False)

transmission
automatic    333032
other         62656
manual        24910
NaN            1695
Name: count, dtype: int64

In [13]:
#for the missing transmission values, i will try to fill with correct transmissions using VIN numbers
missing_trans = vehicles[vehicles['transmission'].isna()][['id', 'VIN', 'transmission']]
missing_trans.head()

,id,VIN,transmission
2961,7312867213,5N1AZ2MHXGN164337,NaN
3537,7305181842,5N1AZ2MHXGN164337,NaN
5088,7316461977,1FDUF5HY1CEB83922,NaN
5250,7315616467,JTHCH96SX60019262,NaN
5284,7315552489,1FDAX57P36ED52397,NaN


In [14]:

#using a free open API to collect the transmission info from the VIN numbers
#vin = '1HGCM82633A004352'
#url = f'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{vin}?format=json'
#response = requests.get(url).json()

#or item in response['Results']:
#    if 'Transmission' in item['Variable']:
#        print(item['Variable'], ":", item['Value'])

In [15]:
#remove rows that have neither transmission or VIN, because i will use VIN to fill out transmission
num_removed = ((vehicles['transmission'].isna()) & (vehicles['VIN'].isna())).sum()
vehicles = vehicles[~(vehicles['transmission'].isna() & vehicles['VIN'].isna())]
print(f"Removed {num_removed} rows with both missing VIN and transmission.")

Removed 5 rows with both missing VIN and transmission.


In [17]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Batch size for the API
BATCH_SIZE = 50

# Filter rows with missing transmission and valid VIN
missing_trans = vehicles[vehicles['transmission'].isna()][['id', 'VIN', 'transmission']].copy()
missing_trans_valid_vins = missing_trans[missing_trans['VIN'].notna()].copy()

# Prepare list of VINs
vin_list = missing_trans_valid_vins['VIN'].tolist()

# Function to get transmission info in batch
def get_transmissions_batch(vins):
    try:
        payload = {'format': 'json', 'data': '\n'.join(vins)}
        url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/'
        response = requests.post(url, data=payload, timeout=15)
        response.raise_for_status()
        results = response.json().get('Results', [])
        
        transmissions = []
        for res in results:
            style = res.get('TransmissionStyle', '').strip()
            speeds = res.get('TransmissionSpeeds', '').strip()
            if style:
                transmissions.append(f"{style} ({speeds} speeds)" if speeds else style)
            else:
                transmissions.append(None)
        return transmissions

    except Exception as e:
        logging.warning(f"Error during batch processing: {e}")

        return [None] * len(vins)

# Collect predictions
predicted_transmissions = []

# Process in batches with progress bar
logging.info(f"Processing {len(vin_list)} VINs in batches of {BATCH_SIZE}...")
for i in tqdm(range(0, len(vin_list), BATCH_SIZE), desc="Decoding VINs"):
    batch_vins = vin_list[i:i + BATCH_SIZE]
    batch_result = get_transmissions_batch(batch_vins)
    
    # Ensure consistent list length even if the function fails
    if not batch_result or len(batch_result) != len(batch_vins):
        logging.warning(f"Batch {i} failed or incomplete. Filling with None.")
        batch_result = [None] * len(batch_vins)
    
    predicted_transmissions.extend(batch_result)

logging.info("Finished batch processing.")

# Assign predictions to matching rows
missing_trans_valid_vins['predicted_transmission'] = predicted_transmissions

# Merge back predicted values into missing_trans
missing_trans = missing_trans.merge(
    missing_trans_valid_vins[['id', 'predicted_transmission']],
    on='id',
    how='left'
)

# Merge into the original vehicles DataFrame and fill in missing values
vehicles = vehicles.merge(
    missing_trans[['id', 'predicted_transmission']],
    on='id',
    how='left'
)

# Fill missing transmissions
vehicles['transmission'] = vehicles['transmission'].fillna(vehicles['predicted_transmission'])

# Drop the helper column
vehicles.drop(columns=['predicted_transmission'], inplace=True)

logging.info("Missing transmission values successfully filled.")
vehicles.head()

2025-04-14 23:13:57,983 - INFO - Processing 1690 VINs in batches of 50...
Decoding VINs: 100%|██████████| 34/34 [00:16<00:00,  2.00it/s]
2025-04-14 23:14:14,962 - INFO - Finished batch processing.
2025-04-14 23:14:15,287 - INFO - Missing transmission values successfully filled.


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7316814884,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,...,NaN,pickup,white,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,NaN,al,32.590,-85.4800,2021-05-04T12:31:18-0500
1,7316814758,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,...,NaN,pickup,blue,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,NaN,al,32.590,-85.4800,2021-05-04T12:31:08-0500
2,7316814989,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,...,NaN,pickup,red,https://images.craigslist.org/01212_jjirIWa0y0...,Carvana is the safer way to buy a car During t...,NaN,al,32.590,-85.4800,2021-05-04T12:31:25-0500
3,7316743432,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,...,NaN,pickup,red,https://images.craigslist.org/00x0x_1y9kIOzGCF...,Carvana is the safer way to buy a car During t...,NaN,al,32.590,-85.4800,2021-05-04T10:41:31-0500
4,7316356412,https://auburn.craigslist.org/cto/d/auburn-uni...,auburn,https://auburn.craigslist.org,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,...,full-size,truck,black,https://images.craigslist.org/00404_l4loxHvdQe...,2013 F-150 XLT V6 4 Door. Good condition. Leve...,NaN,al,32.592,-85.5189,2021-05-03T14:02:03-0500


In [19]:
vehicles['transmission'].value_counts(dropna=False)

transmission
automatic    333032
other         62656
manual        24910
None           1690
Name: count, dtype: int64

In [20]:
vehicles = vehicles.dropna(subset=['transmission'])

In [21]:
vehicles['transmission'].value_counts(dropna=False)

transmission
automatic    333032
other         62656
manual        24910
Name: count, dtype: int64

Fuel type

In [25]:
vehicles['fuel'].value_counts(dropna=False)  #info about fuel types

fuel
gas         351154
other        30526
diesel       29815
hybrid        5132
NaN           2297
electric      1674
Name: count, dtype: int64

In [ ]:
#have 2300 ish NaNs that need to be handeled 